In [6]:
import cv2
import numpy as np
from tensorflow.keras import layers
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.callbacks import Callback, ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import metrics
import tensorflow as tf

In [7]:
def build_model(pretrained):
    model = Sequential([
        pretrained,
        layers.GlobalAveragePooling2D(),
        layers.Dense(1, activation='sigmoid')
    ])
    
    model.compile(
        loss='binary_crossentropy',
        optimizer=Adam(),
        metrics=['accuracy']
    )
    
    return model

In [8]:
# base_path = '../combined-real-and-fake-faces/combined-real-vs-fake/'
base_path = 'C:\\Users\\Deep\\Downloads\\Images\\archive\\real_vs_fake\\real-vs-fake'
image_gen = ImageDataGenerator(rescale=1./255.)

train_flow = image_gen.flow_from_directory(
    base_path + '\\train\\',
    target_size=(224, 224),
    batch_size=32,
    color_mode='grayscale',
    class_mode='binary'
)

valid_flow = image_gen.flow_from_directory(
    base_path + '\\valid\\',
    target_size=(224, 224),
    batch_size=32,
    color_mode='grayscale',
    class_mode='binary'
)


Found 100000 images belonging to 2 classes.
Found 20000 images belonging to 2 classes.


In [9]:
test_flow = image_gen.flow_from_directory(
    base_path + '\\test\\',
    target_size=(224, 224),
    batch_size=1,
    color_mode='grayscale',
    shuffle = False,
    class_mode='binary'
)

Found 20000 images belonging to 2 classes.


In [10]:
densenet = DenseNet121(
    weights=None,
    include_top=False,
    input_shape=(224,224,1)
)
model = build_model(densenet)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 densenet121 (Functional)    (None, 7, 7, 1024)        7031232   
                                                                 
 global_average_pooling2d (G  (None, 1024)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 1)                 1025      
                                                                 
Total params: 7,032,257
Trainable params: 6,948,609
Non-trainable params: 83,648
_________________________________________________________________


In [12]:
train_steps = 100000//32
valid_steps = 20000//32

history = model.fit(
    train_flow,
    epochs = 20,
    steps_per_epoch = train_steps,
    validation_data = valid_flow,
    validation_steps = valid_steps 
)
model.save('grayscale_densenet.h5')

Epoch 1/20
3125/3125 [==============================] - 803s 257ms/step - loss: 0.5202 - accuracy: 0.7407 - val_loss: 0.4659 - val_accuracy: 0.7890
Epoch 2/20
3125/3125 [==============================] - 801s 256ms/step - loss: 0.3013 - accuracy: 0.8719 - val_loss: 0.4400 - val_accuracy: 0.8151
Epoch 3/20
3125/3125 [==============================] - 801s 256ms/step - loss: 0.1747 - accuracy: 0.9306 - val_loss: 0.3892 - val_accuracy: 0.8284
Epoch 4/20
3125/3125 [==============================] - 801s 256ms/step - loss: 0.1126 - accuracy: 0.9573 - val_loss: 0.2795 - val_accuracy: 0.8835
Epoch 5/20
3125/3125 [==============================] - 802s 257ms/step - loss: 0.0758 - accuracy: 0.9717 - val_loss: 0.2109 - val_accuracy: 0.9187
Epoch 6/20
3125/3125 [==============================] - 802s 257ms/step - loss: 0.0559 - accuracy: 0.9796 - val_loss: 0.1112 - val_accuracy: 0.9604
Epoch 7/20
3125/3125 [==============================] - 803s 257ms/step - loss: 0.0417 - accuracy: 0.9851 - val_

In [13]:
model.save('.\\models\\DenseNet_Grayscale_model.h5')

In [14]:
y_pred = model.predict(test_flow)
y_test = test_flow.classes

20000/20000 [==============================] - 354s 18ms/step


In [16]:
print("ROC-AUC Score:", metrics.roc_auc_score(y_test, y_pred))
print("AP Score:", metrics.average_precision_score(y_test, y_pred))
print()
print(metrics.classification_report(y_test, y_pred > 0.5))

ROC-AUC Score: 0.9980781299999999
AP Score: 0.9978361434792907

              precision    recall  f1-score   support

           0       0.99      0.97      0.98     10000
           1       0.97      0.99      0.98     10000

    accuracy                           0.98     20000
   macro avg       0.98      0.98      0.98     20000
weighted avg       0.98      0.98      0.98     20000

